In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.schema import HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda

import json
from langchain.docstore.document import Document


# Data Preparation

## Import scraped data from scraper

In [3]:
from scraper import json_list

In [ ]:
json_list[:5]

## Change json_list to Document

In [31]:
documents = []
for item in json_list:
    content = f"subject: {item['subject']}\n gender: {item['gender']}\n age: {item['age']}\n symptoms: {', '.join(item['symptoms'])}\n question: {item['question']}\n answer: {item['answer']}"
    documents.append(Document(page_content=content, metadata={"source": "scraper"}))

documents[555]

Document(metadata={'source': 'scraper'}, page_content='subject: ขี้หูอุดตัน (Impacted cerumen)\n gender: หญิง\n age: 22\n symptoms: ขี้หูอุดตัน, เสียงดังในหู\n question: มีอาการได้เสียงกรอบแกลบในหูมั้ยมีอาการเจ็บไปโรงพยาบาลกลัวมีแมลงแต่ตรวจเบื้องต้นไม่ผิดปกติแต่กลับมาทีบ้านมีเสียงกรอบแกลบในหูทำให้นอนไม่ค่อยหลับกังวลมากค่ะอาการพึ่งเป็นตอนหลังจากวิ่งเสร็จรบกลัวคุณหมอด้วยค่ะ\n answer: โดยส่วนใหญ่อาการได้ยินเสียงรบกวนในหูมักเกิดจากขี้หูอุดตันหรือมีสิ่งแปลกปลอมติดค้างในหู ถ้าได้ไปตรวจหูที่โรงพยาบาลแล้วไม่พบอะไรแต่อาการยังไม่หาย แนะนำให้ไปตรวจเพิ่มเติมกับแพทย์เ)พาะทางหูคอจมูก เพื่อหาสาเหตุต่อไปครับ')

## Embeding and Store to VectorDB

In [32]:
embedder = OllamaEmbeddings(model="bge-m3")
print(embedder)


base_url='http://localhost:11434' model='bge-m3' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


In [33]:
vector_store = FAISS.from_documents(documents, embedder)
vector_store


In [77]:
retriever = vector_store.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000247739042D0>, search_kwargs={})

# Create chat

In [78]:
llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",
                    temperature=0.4)
print(llm)

model='scb10x/llama3.1-typhoon2-8b-instruct' temperature=0.4


In [79]:
result = llm.invoke("เอกลักษณ์การบวกคืออะไร")
print(result.content)

เอกลักษณ์การบวกหมายถึงสมบัติที่ใช้ในการคำนวณผลรวมของจำนวนสองจำนวนหรือมากกว่า โดยทั่วไปแล้ว เอกลักษณ์การบวกมีหลายรูปแบบ เช่น:

1. 0 + a = a (การเพิ่มศูนย์ไม่เปลี่ยนแปลงค่า)
2. a + 0 = a (การเพิ่มศูนย์ก็ไม่เปลี่ยนแปลงค่าเช่นกัน)
3. a + b = b + a (ลำดับการบวกไม่สำคัญ)

เอกลักษณ์การบวกเหล่านี้ช่วยให้เราสามารถคำนวณผลรวมของจำนวนต่าง ๆ ได้อย่างถูกต้องและสะดวก


In [113]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "คุณคือผู้แพทย์เชี่ยวชาญด้านสุขภาพที่มีความรู้ลึกซึ้งทั้งจากประสบการณ์และฐานข้อมูลทางการแพทย์ "
     "คุณจะต้องให้คำแนะนำที่แม่นยำและปลอดภัย โดยเริ่มต้นด้วยประโยค 'คำตอบจากแพทย์ผู้เชี่ยวชาญ' "
     "จากนั้นให้คำตอบแบ่งเป็นหัวข้อ: สาเหตุที่เป็นไปได้, แนวทางการดูแลเบื้องต้น, คำแนะนำเพิ่มเติม "
     " ใส่ข้อความ 'คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ \nhttps://bit.ly/doctor-refer-agnos' ในช่วงท้ายของการสนทนา"),
    
    ("human", 
     "มีผู้ป่วยที่มีข้อมูลดังนี้:\n"
     "- หัวข้อ: {subject}\n"
     "- เพศ: {gender}\n"
     "- อายุ: {age} ปี\n"
     "- อาการ: {symptoms}\n"
     "- คำถาม: {question}\n"
     "ข้อมูลจากฐานข้อมูลที่เกี่ยวข้อง:\n{context}\n"
     "กรุณาวิเคราะห์และให้คำตอบอย่างละเอียด โดยอ้างอิงจากทั้งความรู้ของคุณและข้อมูลที่ให้มาในส่วนของ context อย่างรอบคอบ"
     "หากข้อมูลใน context ไม่เพียงพอ ให้ใช้ความรู้ทั่วไปของคุณในการตอบคำถามอย่างระมัดระวังและแนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญ"
     )
])


In [114]:
rag_chain = (
    {
        "context": retriever,
        "subject": RunnablePassthrough(),
        "gender": RunnablePassthrough(),
        "age": RunnablePassthrough(),
        "symptoms": RunnablePassthrough(),
        "question": RunnablePassthrough(),
        
    }
    | prompt 
    | llm
    | StrOutputParser()
)

In [103]:
data_input =[ {
    "subject": "อาการปวดหัวเรื้อรัง",
    "gender": "หญิง",
    "age": "35",
    "symptoms": "ปวดหัวบริเวณขมับทั้งสองข้างมาเกือบทุกวัน โดยเฉพาะช่วงบ่าย รู้สึกตึงๆ และมีอาการคลื่นไส้ร่วมด้วย",
    "question": "อาการแบบนี้เกิดจากอะไร และควรดูแลตัวเองอย่างไร?"
},
{
    "subject": "การนอนหลับในผู้สูงอายุ",
    "gender": "ชาย",
    "age": "68",
    "symptoms": "นอนหลับไม่สนิท ตื่นบ่อยตอนกลางคืน และรู้สึกไม่สดชื่นตอนเช้า",
    "question": "ควรปรับพฤติกรรมหรือมีวิธีดูแลสุขภาพอย่างไรให้หลับดีขึ้น?"
},
{
    "subject": "ภาวะขาดวิตามินบี 12 (Vitamin B12 deficiency)",
    "gender": "หญิง",
    "age": "29",
    "symptoms": [
      "เหนื่อยง่าย",
      "มือเท้าชา",
      "เวียนหัว",
      "ลืมง่าย",
      "อ่อนแรง"
    ],
    "question": "ช่วงนี้รู้สึกเหนื่อยง่ายมากค่ะ เดินนิดเดียวก็หอบ มือเท้าชาบ่อย ๆ โดยเฉพาะตอนตื่นนอน รู้สึกเวียนหัวและลืมง่ายขึ้นเรื่อย ๆ อยากทราบว่าอาการแบบนี้เกิดจากอะไร และควรตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคกรดไหลย้อน (GERD)",
    "gender": "ชาย",
    "age": "34",
    "symptoms": [
      "แสบร้อนกลางอก",
      "เรอเปรี้ยว",
      "ไอเรื้อรัง",
      "แน่นหน้าอก",
      "กลืนลำบาก"
    ],
    "question": "ผมมีอาการแสบร้อนกลางอกหลังทานอาหาร โดยเฉพาะอาหารเผ็ดหรือมัน ๆ เรอเปรี้ยวบ่อยมาก และบางครั้งรู้สึกแน่นหน้าอกจนหายใจไม่สะดวก อยากทราบว่าเป็นกรดไหลย้อนหรือเปล่า และควรปรับพฤติกรรมยังไงครับ?"
  },
  {
    "subject": "ภาวะถุงน้ำรังไข่หลายใบ (PCOS)",
    "gender": "หญิง",
    "age": "26",
    "symptoms": [
      "ประจำเดือนมาไม่ปกติ",
      "สิวขึ้นเยอะ",
      "ขนดก",
      "น้ำหนักขึ้นง่าย",
      "ปวดท้องน้อย"
    ],
    "question": "ประจำเดือนมาไม่ตรงเวลาเลยค่ะ บางเดือนก็ไม่มาเลย สิวขึ้นเยอะมากทั้งที่ดูแลผิวดีแล้ว ขนตามตัวก็ดูเยอะขึ้น น้ำหนักขึ้นง่ายมากทั้งที่กินเท่าเดิม อยากทราบว่าอาการแบบนี้เกี่ยวกับฮอร์โมนหรือเปล่า และควรไปตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคภูมิแพ้อากาศ (Allergic Rhinitis)",
    "gender": "ชาย",
    "age": "18",
    "symptoms": [
      "จามบ่อย",
      "คัดจมูก",
      "น้ำมูกไหล",
      "คันจมูก",
      "ตาแดง"
    ],
    "question": "ผมมีอาการจามบ่อยมาก โดยเฉพาะตอนเช้า ๆ หรือเวลาอยู่ในห้องแอร์ น้ำมูกไหล คัดจมูก และบางครั้งตาแดงคันร่วมด้วย อยากทราบว่าเป็นภูมิแพ้อากาศหรือเปล่า และมีวิธีดูแลตัวเองยังไงบ้างครับ?"
  }
]

In [112]:
for i, data in enumerate(data_input):
    result = rag_chain.invoke(data)
    print(f"หัวข้อที่ {i+1}\n")
    print(result)




หัวข้อที่ 1

คำตอบจากแพทย์ผู้เชี่ยวชาญ

สาเหตุที่เป็นไปได้:
อาการปวดหัวบริเวณขมับทั้งสองข้างและคลื่นไส้ที่เกิดขึ้นเกือบทุกวัน อาจเกิดจากการบาดเจ็บที่กล้ามเนื้อคอหรือความเครียด แต่ยังอาจเป็นอาการของไมเกรนก็ได้

แนวทางการดูแลเบื้องต้น:
1. พักผ่อนให้เพียงพอ
2. หลีกเลี่ยงสาเหตุที่กระตุ้นไมเกรน เช่น แสงจ้า เสียงดัง และกลิ่นแรง
3. ใช้ยาที่เหมาะสมในการบรรเทาอาการปวดหัวและคลื่นไส้ หากอาการรุนแรง ควรปรึกษาแพทย์เพื่อรับยาที่ถูกต้อง

คำแนะนำเพิ่มเติม:
การออกกำลังกายอย่างสม่ำเสมอและการทำกิจกรรมที่ช่วยผ่อนคลาย เช่น โยคะหรือการทำสมาธิ สามารถช่วยลดความเครียดและบรรเทาอาการปวดหัวได้

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos
หัวข้อที่ 2

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

สาเหตุที่เป็นไปได้:
อาการนอนไม่หลับในผู้สูงอายุอาจเกิดจากหลายสาเหตุ เช่น ความเครียด, โรคทางจิตเวชบางอย่าง, หรือการเปลี่ยนแปลงของฮอร์โมนในร่างกาย นอกจากนี้ยังอาจเกิดจากการมีพฤติกรรมการนอนที่ไม่เหมาะสม

แนวทางการดูแลเบื้องต้น:
1. จัดห้องนอนให้มีบรรยากาศเงียบสงบ ไม่มีแ

In [115]:
my_question = rag_chain.invoke({    
    "subject": "กรดไหลย้อน อาหารไม่ย่อย",
    "gender": "ชาย",
    "age": "24",
    "symptoms": "แสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอ",
    "question": "มีคืนนึงตอนผมกำลังจะนอนจู่ๆผมก็รู้สึกแสบคอขึ้นมา แล้วพอตื่นมาก็รู้สึกว่าเหมือนมีก้อนอะไรมาจุกที่คอ ผมเลยค้นหาจากอินเตอร์เน็ตก็พบว่าผมเป็นกรดไหลย้อนจึงไปซื้อยาที่ร้านขายยามากินเป็นเวลา 1 อาทิตย์ จนพอผมเริ่มรู้สึกดีขึ้นผมจึงไปกินบุฟเฟ่ต์ชาบูแล้วทีนี้ผมรู้สึกว่าอาหารมันไม่ย่อยเลยแม้แต่ตอนกำลังจะนอนก็รู้สึกได้ว่ามีอาหารอยู่ในท้องจนถึงวัดถัดไป ควรรักษาต่อยังไงดีครับ"
})

print(my_question)

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

สาเหตุที่เป็นไปได้: อาการแสบอก แสบคอ ท้องอืด อาหารไม่ย่อย และจุกคอที่ผู้ป่วยประสบอยู่อาจเกิดจากกรดไหลย้อน (GERD) ซึ่งเป็นภาวะที่กรดในกระเพาะอาหารไหลย้อนขึ้นไปในหลอดอาหาร ทำให้เกิดอาการแสบและไม่สบายตัว

แนวทางการดูแลเบื้องต้น: ผู้ป่วยควรปรับพฤติกรรมการกินโดยการหลีกเลี่ยงอาหารที่กระตุ้นกรดไหลย้อน เช่น อาหารไขมันสูง, อาหารรสเผ็ด, น้ำอัดลม และเครื่องดื่มแอลกอฮอล์ ควรทานอาหารในปริมาณที่พอเหมาะและไม่เร่งรีบ ควรรอให้กระเพาะอาหารย่อยอาหารเสร็จก่อนนอนหลับ

คำแนะนำเพิ่มเติม: หากผู้ป่วยมีอาการกรดไหลย้อนอย่างต่อเนื่อง หรืออาการไม่ดีขึ้นหลังจากการปรับเปลี่ยนพฤติกรรมการกิน ควรไปพบแพทย์เพื่อรับการตรวจวินิจฉัยและรักษาอย่างเหมาะสม

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [116]:
my_question2 = rag_chain.invoke({    
    "subject": "นอนไม่หลับตอนกลางคืน ความคิดฟุ้งซ่านตอนนอน",
    "gender": "ชาย",
    "age": "30",
    "symptoms": "นอนไม่หลับ, ความคิดฟุ้งซ่าน, ได้ยินเพลงในหัวตลอดเวลานอน",
    "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
})

print(my_question2)

คำตอบจากแพทย์ผู้เชี่ยวชาญ:

สาเหตุที่เป็นไปได้:
อาการนอนไม่หลับและความคิดฟุ้งซ่านอาจเกิดจากหลายสาเหตุ เช่น ความเครียด, ภาวะวิตกกังวล, หรือโรคทางจิตเวชบางอย่าง นอกจากนี้ การมีเพลงเล่นในหัวตลอดเวลาอาจเป็นสัญญาณของภาวะการนอนหลับที่ไม่ปกติ

แนวทางการดูแลเบื้องต้น:
1. จัดห้องนอนให้มีบรรยากาศเงียบสงบ ไม่มีแสงเสียงรบกวน และมีอากาศถ่ายเทสะดวก
2. ใช้ห้องนอนเพื่อการนอนเท่านั้น ไม่ใช้ห้องนอนทำกิจกรรมอื่น เช่น ดูทีวี ทำงาน อ่านหนังสือ
3. ก่อนนอน 1 ชั่วโมงควรมีเวลาให้ตัวเองได้พักผ่อนสบายๆ พยายามไม่คิดเรื่องเครียด เรื่องกังวลต่างๆ
4. เข้านอนเมื่อเกิดความง่วงนอนเท่านั้น
5. ถ้าหากเข้านอนไปนาน 15-20 นาทีแล้วยังนอนไม่หลับ ควรลุกขึ้นมาจากที่นอนเพื่อหากิจกรรมเบาๆ ทำก่อน แล้วกลับเข้าไปนอนเมื่อรู้สึกง่วงนอนอีกครั้ง
6. ตื่นนอนให้เป็นเวลาเดียวกันของทุกวัน
7. ออกกำลังกายอย่างสม่ำเสมอ แต่ไม่ควรออกกำลังกายหักโหมก่อนนอนเพราะจะทำให้นอนไม่หลับมากขึ้น
8. ไม่นอนกลางวัน
9. ไม่ดื่มชา กาแฟ หลังเที่ยงวัน
10. ไม่สูบบุหรี่หรือดื่มสุราก่อนนอน

คำแนะนำเพิ่มเติม:
หากปฏิบัติตามแนวทางการดูแลเบื้องต้นแล้วอาการยังไม่ดีขึ้น ควรไปพ